In [2]:
import pymongo
import pandas as pd
import numpy as np
import re
import time
import datetime
import nltk
from datetime import datetime, timedelta
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from bson.objectid import ObjectId

In [3]:
client = pymongo.MongoClient('localhost', 27017)
db = client.db
threads = db.threads
companies = db.companylist
matches = db.matches
chunks = db.chunks
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/user/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [4]:
df = pd.DataFrame.from_records(threads.find({'Label':{'$ne': 0}}))

In [5]:
sir = SentimentIntensityAnalyzer()

In [6]:
sent = df.loc[77,'Title'] + ' ' + df.loc[77,'Body']

In [7]:
sir.polarity_scores(sent)['compound']

0.9459

In [8]:
for i in range(len(df)):
    sent = df.loc[i,'Title'] + ' ' + df.loc[i,'Body']
    score = sir.polarity_scores(sent)['compound']
    if (score < 0 and df.loc[i, 'Label'] == 'bullish') or (score > 0 and df.loc[i,'Label'] == 'bearish'):
        print(str(score) + ':' + df.loc[i, 'Label'])

-0.9077:bullish
-0.7453:bullish
-0.4522:bullish
-0.9222:bullish
0.9274:bearish
0.8258:bearish
0.1027:bearish
-0.6037:bullish
-0.9495:bullish
0.8022:bearish
-0.5267:bullish
-0.5994:bullish
0.3716:bearish
0.8689:bearish
-0.743:bullish
0.4791:bearish
-0.608:bullish
0.2263:bearish
-0.8371:bullish
-0.8185:bullish
0.6326:bearish
0.9801:bearish
-0.0874:bullish
-0.953:bullish
-0.25:bullish
-0.5574:bullish
0.5978:bearish
0.9976:bearish
-0.8658:bullish
-0.4246:bullish
-0.9578:bullish
-0.3182:bullish
-0.1027:bullish
-0.5829:bullish
-0.2313:bullish
-0.8578:bullish
0.7506:bearish
0.7063:bearish
-0.5789:bullish
-0.4939:bullish
0.8266:bearish
-0.3533:bullish
-0.4767:bullish
-0.3249:bullish
-0.1927:bullish
0.4404:bearish
0.323:bearish
-0.7814:bullish
0.296:bearish
-0.7025:bullish
0.9917:bearish
-0.5106:bullish
-0.599:bullish
0.8943:bearish
0.8957:bearish
-0.6369:bullish
0.9042:bearish
-0.5106:bullish
0.8236:bearish
0.9208:bearish
-0.1027:bullish
0.6739:bearish
0.9169:bearish
-0.3972:bullish
-0.1655:bu

In [9]:
import flair
from flair.models import TextClassifier
from flair.data import Sentence

In [10]:
classifier = TextClassifier.load('en-sentiment')
sentence = Sentence(sent)
classifier.predict(sentence)

2020-10-29 15:21:25,056 loading file /home/user/.flair/models/sentiment-en-mix-distillbert_3.1.pt


In [11]:
sentence.labels[0]

NEGATIVE (0.9815)

In [12]:
for i in range(len(df)):
    sent = df.loc[i,'Title'] + ' ' + df.loc[i,'Body']
    sentence = Sentence(sent)
    classifier.predict(sentence)
    print(df.loc[i,'Label'] + ': ' + str(sentence.labels[0]))

bearish: NEGATIVE (0.9999)
bullish: NEGATIVE (0.8538)
bullish: NEGATIVE (1.0)
bullish: POSITIVE (0.9986)
bullish: POSITIVE (0.6076)
bullish: NEGATIVE (0.9998)
bullish: NEGATIVE (0.9994)
bullish: NEGATIVE (0.9999)
bullish: NEGATIVE (0.6174)
bullish: NEGATIVE (0.9998)
bearish: NEGATIVE (1.0)
bullish: POSITIVE (0.849)
bullish: NEGATIVE (0.9989)
bullish: NEGATIVE (0.9463)
bullish: NEGATIVE (0.859)
bullish: NEGATIVE (0.9999)
bearish: NEGATIVE (0.9998)
bullish: NEGATIVE (0.9995)
bearish: NEGATIVE (1.0)
bearish: NEGATIVE (1.0)
bullish: POSITIVE (0.9216)
bullish: NEGATIVE (0.9955)
bullish: NEGATIVE (0.9999)
bearish: NEGATIVE (0.9983)
bullish: NEGATIVE (1.0)
bullish: POSITIVE (0.8317)
bullish: POSITIVE (0.8583)
bullish: NEGATIVE (0.9385)
bullish: NEGATIVE (0.9987)
bullish: NEGATIVE (0.9731)
bullish: NEGATIVE (1.0)
bullish: NEGATIVE (0.9986)
bullish: NEGATIVE (0.9996)
bullish: NEGATIVE (0.9864)


KeyboardInterrupt: 